In [6]:
from happy_hub.models import OllamaModel

In [18]:
model = OllamaModel(
    url="http://34.1.198.45:11434/v1",
    model_config_dict={
        "model": "gemma2:27b",
        "temperature": 0.0,
        "stream": True
    }
)

In [20]:
model.run(messages=[
    {"role": "user", "content": "xin chào"}
])

In [21]:
import yaml # type: ignore

### Semantic Router

In [2]:
with open("../happy_hub/prompts/semantic_router.yml", 'r') as file:
    semantic_router_yaml = yaml.safe_load(file)

In [3]:
system_prompt_template = ""
for key in semantic_router_yaml:
    system_prompt_template += semantic_router_yaml[key]

In [5]:
from happy_hub.functions import RETRIEVAL_FUNCS

In [9]:
examples = [
    {"role": "user", "content": "xin chào"},
    {"role": "assistant",  'content': '{"name": "fun_chat"}'},
    {"role": "user", "content": "quy định nghỉ phép của MB gồm những gì?"},
    {"role": "assistant", "content": '{"name": "mb_info_query"}'},
    {"role": "user", "content": "MB có gì hay ho nhỉ ?"},
    {"role": "assistant", "content": '{"name": "mb_info_query"}'},
]

system_prompt = system_prompt_template.format(
    tools = [func.get_openai_tool_schema() for func in RETRIEVAL_FUNCS],
)

# Prepare the input messages
messages = [
    {"role": "user", "content": system_prompt},
    {"role": "assistant", "content": '{"name": "mb_info_query"}'}
]

messages.extend(examples)

messages.append({
    "role": "user", "content": "làm nhân viên chính thức có được giãm lãi vay vốn"
})

# Run the model with the input messages
response = model.run(messages=messages) # type: ignore

for chunk in response:
    print(chunk.choices[0].delta.content, end="")

{"name": "mb_info_query"}

## Fun Chat

In [3]:
with open("../happy_hub/prompts/fun_chat.yml", 'r') as file:
    fun_chat_yaml = yaml.safe_load(file)

In [4]:
fun_chat_template = ""
for key in fun_chat_yaml:
    fun_chat_template += fun_chat_yaml[key]

In [5]:
# Prepare the input messages
fun_chat_messages = [
    {"role": "user", "content": fun_chat_template},
    {"role": "assistant", "content": 'Rất vui được giúp đỡ bạn, mình là happy hub!'}
]

fun_chat_messages.append({
    "role": "user", "content": "tôi muốn đi chơi hụi để giàu nhanh"
})

# Run the model with the input messages
response = model.run(messages=fun_chat_messages) # type: ignore

for chunk in response:
    print(chunk.choices[0].delta.content, end="")

NameError: name 'model' is not defined

## MB Info Query

In [22]:
with open("../happy_hub/prompts/mb_info_query.yml", 'r') as file:
    mb_info_query_yaml = yaml.safe_load(file)

mb_info_query_template = ""
for key in mb_info_query_yaml:
    mb_info_query_template += mb_info_query_yaml[key]

In [23]:
from happy_hub.storage import SimpleVectorDB

vector_db = SimpleVectorDB()
vector_db.load_from_file("../vector_db.pkl")

In [24]:
def parse_qa_text(text):
    """
    Parses a given text into a dictionary with 'question' and 'answer' keys.

    Args:
        text (str): The input text containing a question and answer separated by ';'.

    Returns:
        dict: A dictionary with 'question' and 'answer' as keys.

    Example:
        text = "question: Quy định về chế độ nghỉ phép có hưởng lương tại MB?; answer: Bạn có thời gian làm việc..."
        result = parse_qa_text(text)
    """
    # Split the text into question and answer parts
    parts = text.split(';')
    
    # Initialize the result dictionary
    result = {}
    
    # Parse the question
    if parts[0].startswith('question:'):
        result['question'] = parts[0].split('question:', 1)[1].strip()
    
    # Parse the answer
    if len(parts) > 1 and parts[1].startswith(' answer:'):
        result['answer'] = parts[1].split(' answer:', 1)[1].strip()
    
    return result

In [25]:
import json
results = vector_db.search("ôi chao ôi tùng", top_k=3)
document = []
for doc, score in results:
    print(parse_qa_text(doc))
    print('-'*30)

{'question': 'utter_ket_thuc', 'answer': 'Happy Hub hẹn gặp lại và chúc bạn một ngày làm việc suôn sẻ'}
------------------------------
{'question': 'Chào hỏi', 'answer': 'Chuyện gì khó, có Happy Hub lo!!! Hãy để mình hỗ trợ bạn nhé'}
------------------------------
{'question': 'Chào hỏi', 'answer': 'Chào bạn, mình là Happy Hub rất vui khi được hỗ trợ bạn. Bạn đang quan tâm tới vấn đề gì? Vui lòng cho mình biết để bên mình hỗ trợ nhé!'}
------------------------------


In [28]:
def answer_mb_info_query(query: str):
    import json
    results = vector_db.search(query=query, top_k=5)
    documents = []
    for doc, score in results:
        documents.append(str(parse_qa_text(doc)))
    system_prompt = mb_info_query_template.format(documents = '\n-----------------------------\n'.join(documents))
    
    # print(system_prompt)
    
    mb_query_messages = [
        {"role": "user", "content": system_prompt},
        {"role": "user", "content": query},
    ]
    response = model.run(mb_query_messages) # type: ignore
    for chunk in response:
        print(chunk.choices[0].delta.content, end="")

In [29]:
answer_mb_info_query("chương trình tín dụng xanh ở MB")

Rất tiếc, hiện tại mình không có đủ thông tin chính xác để trả lời câu hỏi của bạn một cách đầy đủ. Để có được thông tin chi tiết và chính xác nhất, bạn hãy liên hệ với bộ phận nhân sự của MB tại abc.xyz 
